# Minumum Flow Algorith using Cycle Cancelling and SSP

#### Andremo a vedere un paio di algoritmi per il calcolo del flusso minimo, in particolare Cycle Cancelling e SSP


In [4]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, Dropdown, IntText, Checkbox
import csv
import os
import sys
import heapq

#### Qui sotto troviamo un pezzo di codice che ci permetterà di visualizzare a schermo i vari grafi precedenteme creati tramite tramite graphs_generator

In [5]:
file = os.listdir("./graphs/")
file_list = ["./graphs/" + f for f in file]
_ = file_list.pop(0)  # Elimino il primo file che trova (file autogenerato che non fa parte dei grafi)

def fromPathToGraph(path: list):
    G = nx.DiGraph()  # Utilizzo DiGraph per grafo orientato
    for index in range(0, len(path) - 1):
        G.add_edge(path[index], path[index + 1])
    return G

@interact(file=Dropdown(options=file_list, description='Graph:'),
          show=Checkbox(description="Plot the graph"))
def selectGraph(file, show): 
    df = pd.read_csv(file, names=["Node1", "Node2", "Capacità"])
    df = df.iloc[1:]  # Elimina la prima riga
    G = nx.DiGraph()  # Utilizzo DiGraph per grafo orientato
    Graphtype = nx.DiGraph() 
    G = nx.from_pandas_edgelist(df, source="Node1", target="Node2", edge_attr='Capacità', create_using=Graphtype)
    if show:
        pos = nx.spring_layout(G)  # Calcolo la posizione dei nodi
        edge_labels = {(u, v): d["Capacità"] for u, v, d in G.edges(data=True)}  # Creo un dizionario di etichette degli archi
        
        nx.draw(G, pos, with_labels=True, node_size=700, node_color="skyblue", font_size=10)
        nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8)  # Aggiungo etichette degli archi


interactive(children=(Dropdown(description='Graph:', options=('./graphs/grafo_archi_7.csv', './graphs/grafo_ar…

## Cycle cancelling

##### L'algortimo "cycle cancelling" viene utilizzato epr calcolare il flusso minimo in un grafo. Il suo obbiettivo è determinare la quantità minima di flusso che può essere inviata da una sorgente ad una destinazione attraverso un grafo ponderato, rispettando le capacità degli archi e soddisfacendo i vincoli di conservazione del flusso nei nodi intermedi.
##### Il processo dell'algoritmo coinvolge la ricerca di cicli negativi nel grafo residuo, dove il grafo residuo rappresenta le capacità residue degli archi dopo l'invio del flusso corrente. Una volta individuato un ciclo di negativo, il flusso viene aumentato lungo gli archi all'interno del ciclo in modo da ridurre la negatività. Questo processo continua fino a quando non è più possibile trovare cicli negativi nel grafo residuo.
##### L'algoritmo di "cycle cancelling" è una delle tecniche classiche per risolvere il problema del flusso minimo e garantisce la convergenza a una soluzione ottimale. È particolarmente utile quando le capacità degli archi sono variabili o quando si desidera trovare il flusso minimo in un grafo di grandi dimensioni.


#### Qui sotto troviamo il codice che ci permetterà di trovare il flusso di costo minimo partendo da un grafo a nostra scelta. Andremo ad ultilizzare l'algortmo del cycle cancelling offerto da NetworkX.

In [ ]:
# Chiedo all'utente il nome del file CSV contenente il grafo
file_name = input("Inserisci il nome del file CSV contenente il grafo: ")

output_folder_graphs = 'graphs'
graph_file_name = os.path.join(output_folder_graphs, f'grafo_archi_{file_name}.csv')

# Carico il grafo da un file CSV
G = nx.DiGraph()  # Grafo diretto
with open(graph_file_name, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        source = int(row['Nodo1'])
        target = int(row['Nodo2'])
        capacity = int(row['Capacità'])
        G.add_edge(source, target, capacity=capacity)

# Calcolo il flusso minimo utilizzando l'algoritmo del ciclo di cancellazione
flow_value, flow_dict = nx.maximum_flow(G, 0, 9)  # Inserisco i nodi sorgente e destinazione desiderati

# Restituisco il risultato del flusso minimo
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for source_node, target_node in G.edges():
    print(f'Nodo {source_node} -> Nodo {target_node}: {flow_dict[source_node][target_node]}')


##### Qui sotto troviamo una piccola implemetazione di come potrebbe funzionare la funzione già più completa che si trovara in NetworkX. 

In [ ]:
def cycle_canceling(graph, source, sink):
    # Inizializzo il flusso a zero
    flow_value, flow_dict = nx.maximum_flow(graph, source, sink)
    
    # Calcolo il costo residuo (differenza tra costi degli archi e flusso)
    residual_graph = graph.copy()
    for u, v, attr in residual_graph.edges(data=True):
        attr['capacity'] -= flow_dict[u][v]
        attr['cost'] = -attr['cost']
    
    # Ripeto fino a quando ci sono archi con capacità residua positiva e costo negativo
    while True:
        # Trovo un arco con capacità residua positiva e costo negativo
        edge_to_update = None
        for u, v, attr in residual_graph.edges(data=True):
            if attr['capacity'] > 0 and attr['cost'] < 0:
                edge_to_update = (u, v)
                break
        
        # Se non ci sono più archi da aggiornare, esco dal ciclo
        if edge_to_update is None:
            break
        
        # Trovo un cammino dal nodo sorgente al nodo destinazione nel grafo residuo
        try:
            path = nx.shortest_path(residual_graph, source, sink, weight='cost')
        except nx.NetworkXNoPath:
            break
        
        # Calcolo la capacità residua minima lungo il cammino
        min_capacity = min(residual_graph[u][v]['capacity'] for u, v in zip(path, path[1:]))
        
        # Aggiorno il flusso lungo il cammino
        for u, v in zip(path, path[1:]):
            residual_graph[u][v]['capacity'] -= min_capacity
            residual_graph[v][u]['capacity'] += min_capacity
        
        # Calcolo il costo residuo
        for u, v, attr in residual_graph.edges(data=True):
            attr['cost'] = -attr['cost']
        
        # Aggiorno il flusso massimo
        flow_value += min_capacity
    
    return flow_value, flow_dict

# Esempio di utilizzo
graph = nx.DiGraph()

# Aggiungere nodi e archi con capacità e costi
# ...

source = 0
sink = 9

flow_value, flow_dict = cycle_canceling(graph, source, sink)
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for u, v in graph.edges():
    print(f'Nodo {u} -> Nodo {v}: {flow_dict[u][v]}')


# Successive Shortest Path (SSP) 

##### SSP è un algoritmo utilizzato per risolvere il problema del flusso minimo in un grafo di flusso. L'obiettivo del problema del flusso minimo è trovare la quantità minima di flusso che può essere inviata da una sorgente a una destinazione in un grafo diretto ponderato, rispettando le capacità degli archi e minimizzando il costo totale del flusso. L'algoritmo SSP è una variante dell'algoritmo di Ford-Fulkerson ed è particolarmente adatto per risolvere problemi di flusso minimo in cui i costi sugli archi possono essere positivi o negativi.

##### Esempio del funzionamento:

##### 1. Inizia con un flusso iniziale nullo nel grafo di flusso.

##### 2. Finché è possibile trovare un cammino dalla sorgente alla destinazione nel grafo residuo (il grafo originale meno il flusso corrente), continua a iterare.

##### 3. Per trovare il cammino più breve (in termini di costo) dal nodo sorgente al nodo destinazione, l'algoritmo utilizza un algoritmo di ricerca del cammino più breve, come l'algoritmo di Dijkstra o l'algoritmo di Bellman-Ford. La metrica del cammino più breve è il costo totale dei costi sugli archi.

##### 4. Una volta trovato il cammino più breve nel grafo residuo, il flusso viene aumentato lungo questo cammino, riducendo la capacità degli archi in base alla quantità di flusso aggiunto.

##### 5. Si continua questo processo fino a quando non è più possibile trovare un cammino dalla sorgente alla destinazione nel grafo residuo. A questo punto, si ha una soluzione ottimale per il problema del flusso minimo.

##### Qui sotto troviamo il codice che ci permetterà di calcolare il flusso minimo di un grafo partenedo da uno a nostra scelta. Utlizzeremo la funzione SSP implementata da NetworkX.

In [12]:
# Chiedo all'utente il nome del file CSV contenente il grafo
file_name = input("Inserisci il nome del file CSV contenente il grafo: ")

output_folder_graphs = 'graphs'
graph_file_name = os.path.join(output_folder_graphs, f'grafo_archi_{file_name}.csv')

# Carico il grafo da un file CSV
G = nx.DiGraph()  # Grafo diretto
with open(graph_file_name, 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        source = int(row['Nodo1'])
        target = int(row['Nodo2'])
        capacity = int(row['Capacità'])
        G.add_edge(source, target, capacity=capacity)

# Adatto il grafo in modo che tutti i costi siano positivi
for u, v, attr in G.edges(data=True):
    attr['weight'] = -attr['capacity']

# Calcolo il flusso minimo utilizzando l'algoritmo di flusso massimo (Successive Shortest Path)
source = 0  # Nodo sorgente
target = 9  # Nodo destinazione

flow_dict = nx.min_cost_flow(G)

# Ottengo il valore del flusso minimo sommando i flussi uscenti dal nodo sorgente (0)
flow_value = sum(flow_dict[source][neighbor] for neighbor in G.neighbors(source))

# Restituisco il risultato del flusso minimo
print(f'Valore del flusso minimo: {flow_value}')
print('Flusso minimo:')
for u, v in G.edges():
    print(f'Nodo {u} -> Nodo {v}: {flow_dict[u][v]}')

Valore del flusso minimo: 0
Flusso minimo:
Nodo 0 -> Nodo 1: 0
Nodo 0 -> Nodo 2: 0
Nodo 0 -> Nodo 8: 0
Nodo 1 -> Nodo 5: 1
Nodo 1 -> Nodo 6: 2
Nodo 1 -> Nodo 8: 0
Nodo 1 -> Nodo 9: 2
Nodo 2 -> Nodo 1: 3
Nodo 2 -> Nodo 3: 3
Nodo 2 -> Nodo 5: 2
Nodo 2 -> Nodo 8: 0
Nodo 2 -> Nodo 9: 2
Nodo 8 -> Nodo 1: 2
Nodo 8 -> Nodo 4: 2
Nodo 8 -> Nodo 6: 2
Nodo 8 -> Nodo 7: 2
Nodo 5 -> Nodo 8: 3
Nodo 6 -> Nodo 2: 4
Nodo 6 -> Nodo 3: 3
Nodo 6 -> Nodo 7: 4
Nodo 9 -> Nodo 3: 5
Nodo 9 -> Nodo 6: 5
Nodo 3 -> Nodo 2: 6
Nodo 3 -> Nodo 8: 5
Nodo 4 -> Nodo 2: 0
Nodo 4 -> Nodo 5: 0
Nodo 4 -> Nodo 6: 2
Nodo 4 -> Nodo 9: 0
Nodo 7 -> Nodo 9: 6



##### Qui sotto troviamo una piccola implemetazione di come potrebbe funzionare la funzione già più completa che si trovara in NetworkX. 

In [ ]:
class Edge:
    def __init__(self, start, end, capacity, cost):
        self.start = start
        self.end = end
        self.capacity = capacity
        self.cost = cost
        self.flow = 0

def add_edge(graph, start, end, capacity, cost):
    # Aggiungo un arco al grafo, considerando anche l'arco inverso con capacità 0 e costo negativo
    forward_edge = Edge(start, end, capacity, cost)
    backward_edge = Edge(end, start, 0, -cost)  # Arco inverso con capacità 0 e costo negativo
    graph[start].append(forward_edge)
    graph[end].append(backward_edge)

def successive_shortest_path(graph, source, sink):
    n = len(graph)
    INF = sys.maxsize
    dist = [INF] * n
    prev = [None] * n

    max_flow = 0
    min_cost = 0

    while True:
        dist[source] = 0
        hq = [(0, source)]  # Coda prioritaria per l'algoritmo di Dijkstra
        while hq:
            d, u = heapq.heappop(hq)
            if d > dist[u]:
                continue
            for edge in graph[u]:
                if edge.capacity - edge.flow > 0 and dist[edge.end] > dist[u] + edge.cost:
                    dist[edge.end] = dist[u] + edge.cost
                    prev[edge.end] = edge
                    heapq.heappush(hq, (dist[edge.end], edge.end))

        if dist[sink] == INF:
            break

        augmenting_path_flow = INF
        node = sink
        while node != source:
            edge = prev[node]
            augmenting_path_flow = min(augmenting_path_flow, edge.capacity - edge.flow)
            node = edge.start

        max_flow += augmenting_path_flow
        min_cost += augmenting_path_flow * dist[sink]

        node = sink
        while node != source:
            edge = prev[node]
            edge.flow += augmenting_path_flow
            for backward_edge in graph[edge.end]:
                if backward_edge.end == edge.start:
                    backward_edge.flow -= augmenting_path_flow
                    break
            node = edge.start

    return max_flow, min_cost

if __name__ == "__main__":
    # Esempio di utilizzo
    num_nodi = 4
    sorgente = 0
    pozzo = 3
    grafo = [[] for _ in range(num_nodi)]

    add_edge(grafo, 0, 1, 2, 1)
    add_edge(grafo, 0, 2, 1, 2)
    add_edge(grafo, 1, 2, 1, 1)
    add_edge(grafo, 1, 3, 2, 3)
    add_edge(grafo, 2, 3, 2, 1)

    flusso_massimo, costo_minimo = successive_shortest_path(grafo, sorgente, pozzo)
    print(f"Flusso Massimo: {flusso_massimo}")
    print(f"Costo Minimo: {costo_minimo}")


# Conclusioni

##### Possiamo notare come partendo da uno stesso grafo i due algoritmi restituiscano un flusso minimo differente.

##### È possibile che l'algoritmo del cycle cancelling e l'algoritmo Successive Shortest Path (SSP) restituiscano valori diversi per il flusso minimo in un grafo specifico. Questo può essere dovuto a diversi fattori, tra cui la struttura del grafo, i costi associati agli archi e le capacità dei nodi.

##### L'algoritmo del cycle cancelling è specificamente progettato per risolvere il problema del flusso minimo in grafi a costo minimo, mentre SSP può essere utilizzato su una gamma più ampia di problemi di flusso. La differenza nei risultati potrebbe essere dovuta a come ciascun algoritmo gestisce determinate configurazioni di grafo o costi negativi.

##### Se l'obiettivo è ottenere il flusso minimo in un grafo specifico, è importante considerare quale algoritmo è più adatto per il tuo caso specifico e quali parametri e configurazioni stai utilizzando.

##### Ecco alcune ragioni per cui potresti ottenere risultati diversi utilizzando CC e SSP sullo stesso grafo:

##### 1. Inizializzazione del flusso: L'inizializzazione del flusso iniziale può variare tra le implementazioni di CC e SSP. Un'inizializzazione diversa può portare a flussi diversi.

##### 2. Scelta dei cammini: SSP determina il flusso minimo calcolando cammini augmenting successivi, mentre CC si concentra sulla cancellazione di cicli negativi. Le scelte dei cammini possono variare tra i due algoritmi, portando a flussi diversi.

##### 3. Cicli negativi: CC si concentra sulla cancellazione dei cicli negativi all'interno del grafo residuo, mentre SSP calcola cammini augmenting successivi. La scoperta di cicli negativi e la loro cancellazione può variare tra le implementazioni.

##### 4. Precisione numerica: Alcune implementazioni possono gestire la precisione numerica in modo diverso, il che potrebbe influenzare i risultati finali.

##### 5. Condizioni di arresto: Le implementazioni possono avere criteri di arresto diversi per determinare quando il flusso minimo è stato trovato.

##### 6. Strutture dati e ottimizzazioni: Le implementazioni possono utilizzare diverse strutture dati o ottimizzazioni, influenzando l'efficienza e i risultati.